In [1]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation

In [3]:
# if necessary: Select start and end frames for visualisation. 
t_start = 30
t_end = 215

In [4]:
with h5py.File("F:/PhD_Franziska/Headon/Headon_results/mesenchym/video1_slice4/whole_image/U_matrix", "r") as file:
    U = file["matrix"][t_start:t_end]

with h5py.File("F:/PhD_Franziska/Headon/Headon_results/mesenchym/video1_slice4/whole_image/V_matrix", "r") as file:
    V = file["matrix"][t_start:t_end]

with h5py.File("F:/PhD_Franziska/Headon/Headon_results/mesenchym/video1_slice4/whole_image/M_matrix", "r") as file:
    M = file["matrix"][t_start:t_end]
    
with h5py.File("F:/PhD_Franziska/Headon/Headon_results/mesenchym/video1_slice4/whole_image/xgrid_matrix", "r") as file:
    xgrid = file["matrix"][t_start:t_end]
    
with h5py.File("F:/PhD_Franziska/Headon/Headon_results/mesenchym/video1_slice4/whole_image/ygrid_matrix", "r") as file:
    ygrid = file["matrix"][t_start:t_end]

In [6]:
with h5py.File("F:/PhD_Franziska/Headon/Headon_results/mesenchym/video1_slice4/headon_video1_layer4.h5", "r") as file:
    slices = file["data"][t_start:t_end]

# Visualization of original Tissue' video

In [1]:
# fig = plt.figure(figsize = (5, 5), dpi=300)

# def make_frame(i):
#     plt.clf()
#     # Plotting.
#     plt.title("t="+str(i+t_start))
#     plt.imshow(slices[i].T)
#     print(i)

# myanim = FuncAnimation(fig, make_frame, frames = len(U)-1, interval = 500)
# myanim.save("original_slice.mp4", writer="ffmpeg")

# Visualization of original PIV results (Only PIV vectors)

In [3]:
# fig = plt.figure(figsize = (10, 10), dpi=200)
# x = xgrid[0]
# y = ygrid[0]

# def make_frame(i):
#     plt.clf()
#     # Plotting.
#     plt.title("t="+str(i+t_start))
#     plt.quiver(x, y, V[i], -U[i]) # If nescessary use scale to rescale the arrow representation.
#     plt.gca().invert_yaxis()
#     print("ha")

# myanim = FuncAnimation(fig, make_frame, frames = len(U)-1, interval = 500)
# myanim.save("vector_field_v1_s4_total_test.mp4", writer="ffmpeg")

# Averaging of vector fields over sliding time intervals

In [6]:
def vector_avg(U, V, M, t, interval):
    # Compute the averaging vectors and their magnitudes at time t. 
    # U, V and M are the total data sets. 
    # Interval: determins the averaging interval before and after t.

    if t < interval:
        U_mid = np.median(U[0 : t+interval+1], axis = 0) # Median filter
        V_mid = np.median(V[0 : t+interval+1], axis = 0)
        # U_mid = np.mean(U[0:t+interval+1], axis = 0) # Mean filter
        # V_mid = np.mean(V[0:t+interval+1], axis = 0)
        print("1")
    elif len(U)-t < interval:
        U_mid = np.median(U[t-interval : len(U)+1], axis = 0)
        V_mid = np.median(V[t-interval : len(U)+1], axis = 0)
        # U_mid = np.mean(U[t-interval:len(U)+1], axis = 0)
        # V_mid = np.mean(V[t-interval:len(U)+1], axis = 0)
        print("2")
    else:
        U_mid = np.median(U[t-interval : t+interval+1], axis = 0)
        V_mid = np.median(V[t-interval : t+interval+1], axis = 0)
        # U_mid = np.median(U[t-interval : t+interval+1], axis = 0)
        # V_mid = np.median(V[t-interval : t+interval+1], axis = 0)
        print("3")

    # Update the M (magnitude matrix)
    M_mid = np.sqrt(U_mid**2 + V_mid**2)
        
    return U_mid, V_mid, M_mid

## Filter out the extra long vectors
Depending on the vector length how many times of std above the mean length. 

In [11]:
def postprocess(U, V, M, std_factor):
    # Defining the mask.
    meanM = np.mean(M)
    stdM = np.std(M)
    mask = M < meanM + std_factor * stdM
    M *= mask
    # filter the U, V vector components depending on masked M.
    # U[M == 0] = "nan"
    # V[M == 0] = "nan"
    return U, V, M

## Firstly averaging then postprocessing. Make video.

### With background

In [4]:
# fig = plt.figure(figsize = (10, 10))
# x = xgrid[0]
# y = ygrid[0]

# def make_frame(i):
#     plt.clf()

#     # Averaging
#     U_avg, V_avg, M_avg = vector_avg(U, V, M, i, 15)
#     # Postprocessing.
#     #U_post, V_post, _ = postprocess(U_avg, V_avg, M_avg, 3)
#     # Plotting.
#     plt.title("t="+str(i+t_start))
#     plt.imshow(slices[i].T)
#     plt.quiver(x, y, V_avg, -U_avg, color = "royalblue") # can use parameter scale to rescale the arrows.
#     #plt.gca().invert_yaxis()
#     #print("ha")

# myanim = FuncAnimation(fig, make_frame, frames = len(U)-1, interval = 500)
# myanim.save("avg_filter_backgr.mp4", writer="ffmpeg")

### Only vectors

In [5]:
# fig = plt.figure(figsize = (10, 10), dpi=300)
# x = xgrid[0]
# y = ygrid[0]
# t_start = 30

# def make_frame(i):
#     plt.clf()

#     # Averaging (after the space averaging)
#     U_avg, V_avg, M_avg = vector_avg(U, V, M, i, 15) 
#     # # Postprocessing.
#     # U_post, V_post, _ = postprocess(U_avg, V_avg, M_avg, 2)
#     # # Set boundary to be nan. 
#     #U_final, V_final, _ = set_boundary_zero(U_avg, V_avg, M_avg)
#     # Plotting.
#     plt.title("t="+str(i+t_start))
#     plt.quiver(x, y, V_avg, -U_avg, color="royalblue")
#     plt.gca().invert_yaxis()

# myanim = FuncAnimation(fig, make_frame, frames = len(U)-1, interval = 500)
# myanim.save("avg_filter_backgr_only_vector.mp4", writer="ffmpeg")

## Write the U, V, M in h5 files.
Remember to synchronize the averaging interval as above. 

In [6]:
# m_U = []
# m_V = []
# m_M = []

# for i in range(len(U)):
#     # Averaging
#     U_avg, V_avg, M_avg = vector_avg(U, V, M, i, 15)
#     # Postprocessing.
#     #U_post, V_post, M_post = postprocess(U_avg, V_avg, M_avg, 3)
#     m_U.append(U_avg)
#     m_V.append(V_avg)
#     m_M.append(M_avg)

# # Write into h5 file.
# U_avg = h5py.File("U_avg.h5", "w")
# U_avg.create_dataset("matrix", data = m_U)
# U_avg.close()

# V_avg = h5py.File("V_avg.h5", "w")
# V_avg.create_dataset("matrix", data = m_V)
# V_avg.close()

# M_avg = h5py.File("M_avg.h5", "w")
# M_avg.create_dataset("matrix", data = m_M)
# M_avg.close()